In [1]:
%run ./data_creation_utils.ipynb

D:\KISHORE\Binance-Data-Downloader\data\downloaded_data
D:\KISHORE\Binance-Data-Downloader\data\extracted_data
D:\KISHORE\Binance-Data-Downloader\data\downloaded_data
D:\KISHORE\Binance-Data-Downloader\data\extracted_data


C:\Users\Admin\AppData\Local\Temp\ipykernel_39652\3069871549.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(CHROME_DRIVER_PATH)


['1000BTTCUSDT', '1000FLOKIUSDT', '1000LUNCBUSD', '1000LUNCUSDT', '1000PEPEUSDT', '1000SHIBBUSD', '1000SHIBUSDT', '1000XECUSDT', '1INCHUSDT', 'AAVEUSDT', 'ACHUSDT', 'ADABUSD', 'ADAUSDT', 'AGIXBUSD', 'AGIXUSDT', 'AGLDUSDT', 'AKROUSDT', 'ALGOUSDT', 'ALICEUSDT', 'ALPHAUSDT', 'AMBBUSD', 'AMBUSDT', 'ANCBUSD', 'ANCUSDT', 'ANKRUSDT', 'ANTUSDT', 'APEBUSD', 'APEUSDT', 'API3USDT', 'APTBUSD', 'APTUSDT', 'ARBUSDT', 'ARKMUSDT', 'ARKUSDT', 'ARPAUSDT', 'ARUSDT', 'ASTRUSDT', 'ATAUSDT', 'ATOMUSDT', 'AUCTIONBUSD', 'AUDIOUSDT', 'AVAXBUSD', 'AVAXUSDT', 'AXSUSDT', 'BAKEUSDT', 'BALUSDT', 'BANDUSDT', 'BATUSDT', 'BCHUSDT', 'BELUSDT', 'BICOUSDT', 'BLUEBIRDUSDT', 'BLURUSDT', 'BLZUSDT', 'BNBBUSD', 'BNBUSDT', 'BNTUSDT', 'BNXUSDT', 'BNXUSDTSETTLED', 'BTCBUSD', 'BTCBUSD_210129', 'BTCBUSD_210226', 'BTCDOMUSDT', 'BTCSTUSDT', 'BTCUSDT', 'BTCUSDT_210326', 'BTCUSDT_210625', 'BTCUSDT_210924', 'BTCUSDT_211231', 'BTCUSDT_220325', 'BTCUSDT_220624', 'BTCUSDT_220930', 'BTCUSDT_221230', 'BTCUSDT_230331', 'BTCUSDT_230630', 'BTC

data creation utilities successfully initialized


In [2]:
from pathlib import Path
import os
import re
import shutil
from selenium import webdriver
import talib
from datetime import *
from selenium.webdriver.chrome.options import Options
import glob
import zipfile
import pandas as pd
from pandas.api.types import is_numeric_dtype
import inspect
import talib
import time
import numpy as np
import requests
import urllib.request


In [3]:


master_dictionary = {
    "symbols": [
            SYMBOL_ARRAY[SYMBOL_ARRAY.index('BTCUSDT')],
        #     SYMBOL_ARRAY[SYMBOL_ARRAY.index('ETHUSDT')],
        #         SYMBOL_ARRAY[SYMBOL_ARRAY.index('ETHBUSD')],
#                 SYMBOL_ARRAY[SYMBOL_ARRAY.index('1000PEPEUSDT')],
#         SYMBOL_ARRAY[SYMBOL_ARRAY.index('WLDUSDT')],
        #         SYMBOL_ARRAY[SYMBOL_ARRAY.index('BTCBUSD')]
    ],
    "chart_times": [
#             CHART_TIME_ARRAY[CHART_TIME_ARRAY.index('5m')],
            CHART_TIME_ARRAY[CHART_TIME_ARRAY.index('1m')],
#         CHART_TIME_ARRAY[CHART_TIME_ARRAY.index('15m')]
    ],
    "timeperiods": [5, 8, 13, 21, 30, 34, 50, 55, 89, 100, 144, 200, 233],
    "win_percentage":
    0.73,
    "loss_percentage":
    0.4,
    "monthly_or_daily_1_or_2":
    2
}

In [4]:
def download_data_and_concatenate(master_dictionary):
    # Set the start date and end date
    start_date = datetime(2020, 1, 1)
    end_date = datetime(2080, 1, 1)

    # Initialize the list of year-month pairs
    MONTH_ARRAY = []

    # Iterate over the dates from start to end
    date = start_date
    while date < end_date:
        # Format the date as a year-month pair
        year_month = date.strftime("%Y-%m")
        #
        # Add the year-month pair to the list
        MONTH_ARRAY.append(year_month)

        # Increment the date by one month
        date += timedelta(days=31)
    # Get the current date
    now = datetime.now()

    # Format the date as a string in the desired format
    date_string = now.strftime("%Y-%m")


    idx = MONTH_ARRAY.index(date_string)
    MONTH_ARRAY = MONTH_ARRAY[:idx + 1]

    # Get the current date
    now = datetime.now()

    # Get the first day of the current month
    first_day = now.replace(day=1)

    # Create an empty list to store the days
    DAY_ARRAY = []

    # Loop through the days from the first day of the current month to today
    while first_day <= now:
        # Format the date as a string in the desired format
        date_string = first_day.strftime("%Y-%m-%d")

        # Add the date string to the list
        DAY_ARRAY.append(date_string)

        # Move to the next day
        first_day += timedelta(days=1)
        
        
        
        
#     ####ONLY FOR TESTING PURPOSES######
#     DAY_ARRAY = [
#         '2023-08-01', '2023-08-02', '2023-08-03', '2023-08-04', '2023-08-05',
#         '2023-08-06', '2023-08-07'
#     ]
#     ####ONLY FOR TESTING PURPOSES######
    
    
    
    
    for symbol in master_dictionary["symbols"]:
        for chart_time in master_dictionary["chart_times"]:
            print(f"setting up things for {symbol},{chart_time}")

            # Set up an empty list for the data frames
            df_list = []

            # Compile the regular expression pattern
            pattern = re.compile(f"^{symbol}-{chart_time}-\d{{4}}-\d{{2}}\.zip$")

            # Compile the regular expression pattern for daily zip files
            pattern_daily = re.compile(
                f"^{symbol}-{chart_time}-\d{{4}}-\d{{2}}-\d{{2}}\.zip$")

            # Create the new folder path for daily ZIP files
            new_daily_zip_folder_path = os.path.join(
                download_dir, f"{symbol}-{chart_time}-daily_data")

            # Create the new folder path for ZIP files
            new_zip_folder_path = os.path.join(
                download_dir, f"{symbol}-{chart_time}-monthly_data")

            # Create the new folder path for CSV files
            new_csv_folder_path = os.path.join(output_dir,
                                               f"{symbol}-{chart_time}")

            # Set the file name
            concatenated_file_name = f"{symbol}-{chart_time}.csv"

            # Construct the file path
            concatenated_file_path = os.path.join(new_csv_folder_path,
                                                  concatenated_file_name)

            if not Path(concatenated_file_path).exists():

                download_monthly_data(month_array=MONTH_ARRAY, symbol=symbol, chart_time=chart_time)
                download_daily_data(day_array=DAY_ARRAY, symbol=symbol, chart_time=chart_time)


                # Iterate over the files in the new zip folder
                for file in os.listdir(new_zip_folder_path):
                    # Check if the file matches the pattern
                    if pattern.match(file):
                        # Construct the file path
                        file_path = os.path.join(new_zip_folder_path, file)

                        # Extract the ZIP file
                        with zipfile.ZipFile(file_path, "r") as zip_ref:
                            zip_ref.extractall(new_csv_folder_path)

                        # Construct the CSV file path
                        csv_file_path = os.path.join(
                            new_csv_folder_path,
                            f"{symbol}-{chart_time}{file[-12:-4]}.csv")

                        # Read the CSV file into a data frame, ignoring the headers
                        df = pd.read_csv(csv_file_path, header=None)

                        # Remove the first row (which contains the header)
                        df = df.iloc[1:]

                        # Add it to the list
                        df_list.append(df)

                # Iterate over the files in the new daily zip folder
                for file in os.listdir(new_daily_zip_folder_path):
                    # Check if the file matches the pattern
                    if pattern_daily.match(file):
                        # Construct the file path
                        file_path = os.path.join(new_daily_zip_folder_path, file)

                        # Extract the ZIP file
                        with zipfile.ZipFile(file_path, "r") as zip_ref:
                            zip_ref.extractall(new_csv_folder_path)
                        # Construct the CSV file path
                        csv_file_path = os.path.join(
                            new_csv_folder_path,
                            f"{symbol}-{chart_time}-{file.split('-')[-3]}-{file.split('-')[-2]}-{file.split('-')[-1][:-4]}.csv"
                        )
                        # Read the CSV file into a data frame, ignoring the headers
                        df = pd.read_csv(csv_file_path, header=None)

                        # Remove the first row (which contains the header)
                        df = df.iloc[1:]

                        # Add it to the list
                        df_list.append(df)
                # Concatenate the data frames in the list
                df_final = pd.concat(df_list, ignore_index=True)

                # Read the headers from the first CSV file
                headers = pd.read_csv(csv_file_path, nrows=1).columns

                # Set the headers as the column names of the final dataframe
                df_final.columns = headers

                # Convert 'open_time' and 'close_time' columns to datetime
                df_final['open_time'] = pd.to_datetime(df_final['open_time'], unit='ms').dt.tz_localize(
                    'UTC').dt.tz_convert('Asia/Kolkata')
                df_final['close_time'] = pd.to_datetime(df_final['close_time'], unit='ms').dt.tz_localize(
                    'UTC').dt.tz_convert('Asia/Kolkata')

                # Delete the 'ignore' column
                df_final = df_final.drop(['ignore'], axis=1)

                # Add a new column called 'entry' that will take previous close
                df_final['entry'] = df_final['close'].shift(1)

                # Write the data frame to the Excel file
                df_final.to_csv(concatenated_file_path, index=False)

                directory_final = Path(
                    concatenated_file_path).parent  # Get the parent directory

                # deleting all the other csvs
                for file_path in directory_final.iterdir():
                    if file_path != Path(concatenated_file_path):
                        if file_path.is_file():
                            file_path.unlink()
            else:
                print(f"setting up things only for incremental data for {symbol}{chart_time}")
                df_list = []
                df = pd.read_csv(concatenated_file_path, header=None)
                last_record = df.iloc[-1:]
                df_list.append(last_record)

                df = pd.read_csv(concatenated_file_path)
                df['open_time'] = pd.to_datetime(df['open_time'])
                last_record = df.iloc[-1:]


                # Check if the time part of 'open_time' is "05:15:00+05:30"
                if last_record['open_time'].dt.strftime(
                        '%H:%M:%S%z').item() == "05:15:00+0530":
                    # Calculate the last processed date by subtracting 1 day from the date part
                    last_processed_date = (
                        last_record['open_time'].dt.date -
                        pd.DateOffset(days=1)).item().strftime('%Y-%m-%d')
                    last_processed_date = datetime.strptime(
                        last_processed_date, "%Y-%m-%d")
                    print(last_processed_date)
                    # Get today's date
                    today = datetime.today()
                    print(today)

                    # Initialize DAY_ARRAY
                    DAY_ARRAY = []
                    # Start from the day after last_processed_date
                    current_day = last_processed_date + timedelta(days=1)

                    while current_day <= today:
                        # Format the date as a string in the desired format
                        date_string = current_day.strftime("%Y-%m-%d")

                        # Add the date string to DAY_ARRAY
                        DAY_ARRAY.append(date_string)

                        # Move to the next day
                        current_day += timedelta(days=1)
                    print(DAY_ARRAY)
                    download_daily_data(day_array=DAY_ARRAY, symbol=symbol, chart_time=chart_time)
                    # Iterate over the files in the new daily zip folder
                    for file in Path(new_daily_zip_folder_path).iterdir():
                        # Check if the file matches the pattern
                        if pattern_daily.match(file.name):
                            # Extract the date part from the file name (e.g., "2023-08-19")
                            zip_date = file.name.split('-')[-3:]
                            zip_date_str = '-'.join(zip_date).replace('.zip', '')

                            # Check if the date is in DAY_ARRAY
                            if zip_date_str in DAY_ARRAY:
                                # Construct the file path
                                file_path = file

                                # Extract the ZIP file
                                with zipfile.ZipFile(file_path, "r") as zip_ref:
                                    zip_ref.extractall(new_csv_folder_path)

                                # Construct the CSV file path
                                csv_file_path = Path(new_csv_folder_path) / f"{symbol}-{chart_time}-{zip_date_str}.csv"

                                # Read the CSV file into a data frame, ignoring the headers
                                df = pd.read_csv(csv_file_path, header=None)

                                # Remove the first row (which contains the header)
                                df = df.iloc[1:]

                                # Add it to the list
                                df_list.append(df)
                    # Concatenate the data frames in the list
                    df_final = pd.concat(df_list, ignore_index=True)

                    # Read the headers from the first CSV file
                    headers = pd.read_csv(concatenated_file_path, nrows=1).columns

                    # Set the headers as the column names of the final dataframe
                    df_final.columns = headers

                    # Convert 'open_time' and 'close_time' columns to datetime
                    df_final['open_time'] = pd.to_datetime(df_final['open_time'], unit='ms').dt.tz_localize(
                    'UTC').dt.tz_convert('Asia/Kolkata')
                    df_final['close_time'] = pd.to_datetime(df_final['close_time'], unit='ms').dt.tz_localize(
                        'UTC').dt.tz_convert('Asia/Kolkata')


                    # Add a new column called 'entry' that will take previous close
                    df_final['entry'] = df_final['close'].shift(1)
                    # Append the data frame to the existing CSV file in append mode without headers
                    df_final[1:].to_csv(concatenated_file_path, mode='a', header=False, index=False)
                    directory_final = Path(
                    concatenated_file_path).parent  # Get the parent directory

                    # deleting all the other csvs
                    for file_path in directory_final.iterdir():
                        if file_path != Path(concatenated_file_path):
                            if file_path.is_file():
                                file_path.unlink()


                else:
                    print("Something is wrong with data")
    return ("data downloaded and concatenated")

In [5]:
download_data_and_concatenate(master_dictionary)

setting up things for BTCUSDT,1m
Monthly Data Downloaded for BTCUSDT,1m
Daily Data Downloaded for BTCUSDT,1m


'data downloaded and concatenated'

## Running again below cell just to see if it is downloading anything again

In [6]:
download_data_and_concatenate(master_dictionary)

setting up things for BTCUSDT,1m
setting up things only for incremental data for BTCUSDT1m


C:\Users\Admin\AppData\Local\Temp\ipykernel_39652\2686771108.py:185: DtypeWarning: Columns (1,2,3,4,5,7,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(concatenated_file_path, header=None)


Something is wrong with data


'data downloaded and concatenated'

In [18]:
calculate_wins_losses(master_dictionary=master_dictionary,
                      win_perc=master_dictionary["win_percentage"],
                      loss_perc=master_dictionary["loss_percentage"]
                     )

calculating for BTCUSDT1m


'calculated wins and losses '

In [19]:
calculate_indicator_values_new(master_dictionary=master_dictionary, monthly_or_daily_1_2=master_dictionary["monthly_or_daily_1_or_2"])

open_time                  object
open                      float64
high                      float64
low                       float64
close                     float64
volume                    float64
close_time                 object
quote_volume              float64
count                       int64
taker_buy_volume          float64
taker_buy_quote_volume    float64
entry                     float64
if_short                    int64
if_long                     int64
long_target               float64
short_target              float64
long_stop_loss            float64
short_stop_loss           float64
shorts_win_after          float64
longs_win_after           float64
dual_loss                   int64
entered_before            float64
dtype: object


C:\Users\Admin\AppData\Local\Temp\ipykernel_39652\79161319.py:143: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_columns[f'MIDPOINT_{timeperiod}'] = talib.MIDPOINT(df['close'], timeperiod=timeperiod)
C:\Users\Admin\AppData\Local\Temp\ipykernel_39652\79161319.py:144: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_columns[f'MIDPRICE_{timeperiod}'] = talib.MIDPRICE(df['high'], df['low'], timeperiod=timeperiod)
C:\Users\Admin\AppData\Local\Temp\ipykernel_39652\79161319.py:145: PerformanceWarning: DataFrame is highly fragmente

C:\Users\Admin\AppData\Local\Temp\ipykernel_39652\79161319.py:164: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_columns[f'ROC_{timeperiod}'] = talib.ROC(df['close'], timeperiod=timeperiod)
C:\Users\Admin\AppData\Local\Temp\ipykernel_39652\79161319.py:165: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_columns[f'ROCP_{timeperiod}'] = talib.ROCP(df['close'], timeperiod=timeperiod)
C:\Users\Admin\AppData\Local\Temp\ipykernel_39652\79161319.py:166: PerformanceWarning: DataFrame is highly fragmented.  This is usually the resu

C:\Users\Admin\AppData\Local\Temp\ipykernel_39652\79161319.py:151: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_columns[f'AROON_up_{timeperiod}'], new_columns[f'AROON_down_{timeperiod}'] = talib.AROON(df['high'], df['low'],
C:\Users\Admin\AppData\Local\Temp\ipykernel_39652\79161319.py:151: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_columns[f'AROON_up_{timeperiod}'], new_columns[f'AROON_down_{timeperiod}'] = talib.AROON(df['high'], df['low'],
C:\Users\Admin\AppData\Local\Temp\ipykernel_39652\79161319.py:153: Performan

C:\Users\Admin\AppData\Local\Temp\ipykernel_39652\79161319.py:135: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_columns[f'BB_upper_{timeperiod}'], new_columns[f'BB_middle_{timeperiod}'], new_columns[
C:\Users\Admin\AppData\Local\Temp\ipykernel_39652\79161319.py:135: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_columns[f'BB_upper_{timeperiod}'], new_columns[f'BB_middle_{timeperiod}'], new_columns[
C:\Users\Admin\AppData\Local\Temp\ipykernel_39652\79161319.py:135: PerformanceWarning: DataFrame is highly fragmented.  This

...

C:\Users\Admin\AppData\Local\Temp\ipykernel_39652\79161319.py:160: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_columns[f'MINUS_DM_{timeperiod}'] = talib.MINUS_DM(df['high'], df['low'], timeperiod=timeperiod)
C:\Users\Admin\AppData\Local\Temp\ipykernel_39652\79161319.py:161: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_columns[f'MOM_{timeperiod}'] = talib.MOM(df['close'], timeperiod=timeperiod)
C:\Users\Admin\AppData\Local\Temp\ipykernel_39652\79161319.py:162: PerformanceWarning: DataFrame is highly fragmented.  This i

C:\Users\Admin\AppData\Local\Temp\ipykernel_39652\79161319.py:140: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_columns[f'MA_{timeperiod}'] = talib.MA(df['close'], timeperiod=timeperiod)
C:\Users\Admin\AppData\Local\Temp\ipykernel_39652\79161319.py:143: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_columns[f'MIDPOINT_{timeperiod}'] = talib.MIDPOINT(df['close'], timeperiod=timeperiod)
C:\Users\Admin\AppData\Local\Temp\ipykernel_39652\79161319.py:144: PerformanceWarning: DataFrame is highly fragmented.  This is usually th

C:\Users\Admin\AppData\Local\Temp\ipykernel_39652\79161319.py:167: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_columns[f'ROCR100_{timeperiod}'] = talib.ROCR100(df['close'], timeperiod=timeperiod)
C:\Users\Admin\AppData\Local\Temp\ipykernel_39652\79161319.py:168: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_columns[f'RSI_{timeperiod}'] = talib.RSI(df['close'], timeperiod=timeperiod)
C:\Users\Admin\AppData\Local\Temp\ipykernel_39652\79161319.py:170: PerformanceWarning: DataFrame is highly fragmented.  This is usually th

C:\Users\Admin\AppData\Local\Temp\ipykernel_39652\79161319.py:151: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_columns[f'AROON_up_{timeperiod}'], new_columns[f'AROON_down_{timeperiod}'] = talib.AROON(df['high'], df['low'],
C:\Users\Admin\AppData\Local\Temp\ipykernel_39652\79161319.py:151: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_columns[f'AROON_up_{timeperiod}'], new_columns[f'AROON_down_{timeperiod}'] = talib.AROON(df['high'], df['low'],
C:\Users\Admin\AppData\Local\Temp\ipykernel_39652\79161319.py:153: Performan

'indicators are added to the csv'

## Test cells below

In [7]:
file_path = "D:\KISHORE\Binance-Data-Downloader\data\extracted_data\BTCUSDT-1m\BTCUSDT-1m.csv"

# Use pandas to read the CSV file into a DataFrame
test_df = pd.read_csv(file_path)

In [11]:
test_df.dtypes

open_time                  object
open                      float64
high                      float64
low                       float64
close                     float64
volume                    float64
close_time                 object
quote_volume              float64
count                       int64
taker_buy_volume          float64
taker_buy_quote_volume    float64
entry                     float64
dtype: object

In [16]:
column_name = 'open_time'

# Initialize an empty dictionary to store the sample values
sample_values_dict = {}

# Get the unique data types in the specified column
unique_data_types = test_df[column_name].apply(type).unique()
print(unique_data_types)
for data_type in unique_data_types:
    if data_type == type(pd.to_datetime('')):
        # Handle datetime objects
        values_of_type = test_df[test_df[column_name].apply(lambda x: isinstance(x, pd.Timestamp))][column_name]
    else:
        # Handle other data types
        values_of_type = test_df[test_df[column_name].apply(type) == data_type][column_name]

    # Take up to 5 sample values for the current data type
    sample_values = values_of_type.sample(min(5, len(values_of_type))).tolist()

    # Add the data type as the key and the sample values as the value in the dictionary
    sample_values_dict[str(data_type)] = sample_values

# The sample_values_dict now contains sample values for each unique data type in the specified column
print(sample_values_dict)







[<class 'str'>]
{"<class 'str'>": ['2022-03-28 04:01:00+05:30', '2020-02-19 10:56:00+05:30', '2020-02-08 20:02:00+05:30', '2021-08-13 21:31:00+05:30', '2020-01-19 14:26:00+05:30']}


In [17]:
column_name = 'open_time'

# Get the unique data types in the specified column
unique_data_types = test_df[column_name].apply(type).unique()

for data_type in unique_data_types:
    if data_type == type(pd.to_datetime('')):
        # Handle datetime objects
        values_of_type = test_df[test_df[column_name].apply(lambda x: isinstance(x, pd.Timestamp))][column_name]
    else:
        # Handle other data types
        values_of_type = test_df[test_df[column_name].apply(type) == data_type][column_name]

    # Print the unique values causing the data type
    unique_values = values_of_type.unique()
    print(f"Data Type: {data_type}")
    print(f"Unique Values: {unique_values}")
    print("------------------")

Data Type: <class 'str'>
Unique Values: ['2020-01-01 05:31:00+05:30' '2020-01-01 05:32:00+05:30'
 '2020-01-01 05:33:00+05:30' ... '2023-10-02 05:27:00+05:30'
 '2023-10-02 05:28:00+05:30' '2023-10-02 05:29:00+05:30']
------------------
